In [2]:
import tensorflow as tf
import numpy as np
from tfUtils import *
import matplotlib.pyplot as plt
import innvestigate

Using TensorFlow backend.


In [3]:
from keras.layers import Input, Activation, Dense, Flatten, ZeroPadding2D, Conv2D, BatchNormalization, MaxPooling2D
from keras.models import Model

In [7]:
def generate_data(m, n_x, seed = 9):
    np.random.seed = seed
    X = np.random.randn(m, n_x)
    Y = np.zeros((m, 2))
    dist = np.random.randint(low = 0, high = n_x-1, size = m)
    for i in range(m):
        t = np.random.rand();
        if t < 0.5:
            X[i, dist[i]] = -20
            Y[i, 0] = 1
        elif t >= 0.5:
            X[i, dist[i]] = 20
            Y[i, 1] = 1
    return X, Y, dist

In [ ]:
def initialize_parameters(shape, name, which):
    if which == "weights":
        return tf.get_variable(name, shape, 
                               initializer = tf.contrib.layers.xavier_initializer())
    elif which == "bias":
        return tf.get_variable(name, shape, 
                              initializer = tf.zeros_initializer())
    return

In [9]:
def create_placeholders(n_h, n_w, n_c, n_y):
    X = tf.placeholder(tf.float32, [None, n_h, n_w, n_c])
    Y = tf.placeholder(tf.float32, [None, n_y])
    return X, Y

In [ ]:
def flatten(layer):
    size = layer.shape
    num_features = size[1] * size[2] * size[3]
    flattened_layer = tf.reshape(layer, [-1, num_features])
    return flattened_layer, num_features

In [ ]:
def fully_connected(layer, input_features, output_features, activation, indicator):
    weights = initialize_parameters([input_features, output_features], "W" + str(indicator), "weights")
    bias = initialize_parameters([1, output_features], "b" + str(indicator), "bias")
    Z = tf.matmul(layer, weights) + bias
    if activation == "relu":
        return tf.nn.relu(Z), weights, bias
    elif activation == "sigmoid":
        return tf.nn.sigmoid(Z), weights, bias
    elif activation == "tanh":
        return tf.nn.tanh(Z), weights, bias
    elif activation == "linear":
        return Z, weights, bias

In [ ]:
def conv_2D(layer, filter_size, output_channels, stride, padding, activation, use_maxpool, indicator):
    input_channels = layers.shape[3]
    weights = initialize_parameters([filter_size, filter_size, input_channels, output_channels], "W" + str(indicator)
                                   "weights")
    bias = initialize_parameters([output_channels], "b" + str(indicator), "bias")
    Z = tf.nn.conv2d(layer, weights, strides = [1, stride, stride, 1], padding = padding) + bias
    if activation == "relu":
        A = tf.nn.relu(Z)
    elif activation == "sigmoid":
        A = tf.nn.sigmoid(Z)
    elif activation == "tanh":
        A = tf.nn.tanh(Z)
    elif activation == "linear":
        A = Z
    if use_maxpool:
        return tf.nn.max_pool(A, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID'), weights, bias
    else:
        return A, weights, bias

In [6]:
def compute_cost(Z, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z, labels = Y))
    return cost

In [8]:
X_train, Y_train, dist = generate_data(50000, 144)
X_train = X_train.reshape(-1, 12, 12, 1)